# Advanced Retrieval RAG chatbot on Bharatiya Nyaya Sanhita

## Setting up the environment

In [ ]:
!pip install pypdf

In [ ]:
!pip install langchain==0.0.174

In [ ]:
!pip install langchain-chroma

In [ ]:
!pip install langchain-community

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install chromadb

In [ ]:
!curl https://ollama.ai/install.sh | sh

In [ ]:
!neofetch

In [8]:
!pip install -qU langchain-huggingface

In [9]:
import subprocess
import time

# Start ollama as a backrgound process
command = "nohup ollama serve&"

# Use subprocess.Popen to start the process in the background
process = subprocess.Popen(command,
                            shell=True,
                           stdout=subprocess.PIPE,
                           stderr=subprocess.PIPE)
print("Process ID:", process.pid)
# Let's use fly.io resources
#!OLLAMA_HOST=https://ollama-demo.fly.dev:443
time.sleep(5)  # Makes Python wait for 5 seconds

Process ID: 5953


In [10]:
!ollama -v


ollama version is 0.3.8


In [ ]:
!ollama pull llama3

## Feeding in the document

We feed in a document of Bharatiya Nyaya Sanhita, containing all criminal laws of India - a revised version of Indian Penal Code (IPC).

https://prsindia.org/files/bills_acts/bills_parliament/2023/Bharatiya_Nyaya_Sanhita,_2023.pdf

In [63]:
from pypdf import PdfReader

reader = PdfReader("/content/Bharatiya_Nyaya_Sanhita,_2023.pdf")
pdf_texts = [p.extract_text().strip() for p in reader.pages]

# Filter the empty strings
pdf_texts = [text for text in pdf_texts if text]

print((pdf_texts[0]))

# from langchain_community.document_loaders import PDFPlumberLoader
# loader = PDFPlumberLoader("/content/Bharatiya_Nyaya_Sanhita,_2023.pdf")
# docs = loader.load()

# from pypdf import PdfReader

# reader = PdfReader("/content/Bharatiya_Nyaya_Sanhita,_2023.pdf")
# pdf_texts = [p.extract_text().strip() for p in reader.pages]

# # Filter the empty strings
# pdf_texts = [text for text in pdf_texts if text]

THE BHARATIYA NYAYA SANHITA, 2023
—————
ARRANGEMENT OF CLAUSES
—————
CHAPTER IP
RELIMINARY
CLAUSES
1. Short title, commencement and application.
2. Definitions.
3. General Explanations and expressions.
CHAPTER II
OF PUNISHMENTS
4. Punishments.
5. Commutation of sentence of death or imprisonment for life.
6. Fractions of terms of punishment.7. Sentence may be (in certain cases of imprisonment) wholly or partly rigorous or
simple.
8. Amount of fine, liability in default of payment of fine, etc.9. Limit of punishment of offence made up of several offences.
10. Punishment of person guilty of one of several offences, the judgment stating that it
is doubtful of which.
11. Solitary confinement.12. Limit of solitary confinement.13. Enhanced punishment for certain offences after previous conviction.
CHAPTER III
G
ENERAL  EXCEPTIONS
14. Act done by a person bound, or by mistake of fact believing himself bound, by law.15. Act of Judge when acting judicially.
16. Act done pursuant to the judgment 

## Splitting, chunking and forming embeddings

In [65]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter

character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=1000,
    chunk_overlap=0
)
character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))

print((character_split_texts[10]))
print(f"\nTotal chunks: {len(character_split_texts)}")

129. Punishment for assault or criminal force otherwise than on grave provocation.
130. Assault or criminal force to deter public servant from discharge of his duty.
131. Assault or criminal force with intent to dishonor person, otherwise than on grave
provocation.
132. Assault or criminal force in attempt to commit theft of property carried by a person.
133. Assault or criminal force in attempt wrongfully to confine a person.
134. Assault or criminal force on grave provocation.
Of  Kidnapping, Abduction, Slavery and Forced Labour
135. Kidnapping.
136. Abduction.
137. Kidnapping or maiming a child for purposes of begging.
138. Kidnapping or abducting in order to murder or for ransom etc.
139. Importation of girl or boy from foreign country.
140. W rongfully concealing or keeping in confinement, kidnapped or abducted person .
141. Trafficking of person.
142. Exploitation of a trafficked person .CLAUSES

Total chunks: 558


In [66]:
token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)

token_split_texts = []
for text in character_split_texts:
    token_split_texts += token_splitter.split_text(text)

print((token_split_texts[10]))
print(f"\nTotal chunks: {len(token_split_texts)}")

force. 127. criminal force. 128. assault.

Total chunks: 560


In [67]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

embedding_function = SentenceTransformerEmbeddingFunction()
print(embedding_function([token_split_texts[10]]))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[-0.013619152829051018, 0.04207412526011467, -0.046501994132995605, 0.00871409010142088, -0.05017060786485672, 0.04638867452740669, -0.006798196118324995, -0.008451689034700394, -0.05401458963751793, 0.03464439883828163, 0.11186734586954117, -0.055345118045806885, 0.028954394161701202, 0.03301989287137985, -0.07606334239244461, -0.07169773429632187, 0.06854144483804703, -0.03927399590611458, 0.05108104273676872, 0.11075495928525925, 0.04194996505975723, 0.07057356834411621, 0.09054084122180939, -0.027223195880651474, -0.09071942418813705, 0.006933515891432762, -0.08921657502651215, 0.02356128767132759, -0.04572782665491104, -0.11901222169399261, 0.011740420944988728, 0.04497673735022545, 0.06563227623701096, 0.05679904669523239, -0.008386336266994476, -0.02796681597828865, 0.07263123989105225, -0.02666163444519043, -0.030900493264198303, 0.011999569833278656, -0.02734563872218132, 0.007786242291331291, 0.07630109786987305, 0.059679750353097916, 0.002953886752948165, 0.0034313558135181

In [70]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("BNS", embedding_function=embedding_function)

ids = [str(i) for i in range(len(token_split_texts))]

chroma_collection.add(ids=ids, documents=token_split_texts)
chroma_collection.count()

560

## Seeing extracted documents for normal retrieval

In [129]:
query = "Which clause is related to dowry death?"

results = chroma_collection.query(query_texts=[query], n_results=5)
retrieved_documents = results['documents'][0]

for document in retrieved_documents:
    print((document))
    print('\n')

otherwise than under normal circumstances within seven years of her marriage and it isshown that soon before her death she was subjected to cruelty or harassment by her husbandor any relative of her husband for, or in connection with, any demand for dowry, such deathshall be called “ dowry death ”, and such husband or relative shall be deemed to have causedher death. explanation. — for the purposes of this sub - section, “ dowry ” shall have the same meaning as in section 2 of the dowry prohibition act, 1961. ( 2 ) whoever commits dowry death shall be punished with imprisonment for a term which shall not be less than seven years but which may extend to imprisonment for life. 80. every man who by deceit causes any woman who is not lawfully married to him to believe that she is lawfully married to him and to cohabit or have sexual intercourse with him in that belief, shall be punished with imprisonment of either description for a term which may


extend to ten years, and shall also be li

In [130]:
type(retrieved_documents)

list

In [131]:
def rag(retrieved_documents, question, llm):
  context = ''
  for i in retrieved_documents:
    context += i
  # context = retrieved_documents[0] + retrieved_documents[1] + retrieved_documents[2] + retrieved_documents[3] + retrieved_documents[4]
  prompt = f"""
  You are a chatbot trained on Bharatiya Nyaya Sanhita, the new version of Indian criminal laws. Users will ask you questions related to criminal laws. If you don't know the answer, just say that "I don't know".
  Don't jump to conclusions. Support your answer with proper reasoning always.

  DO NOT USE ANY PRE-EXISTING KNOWLEDGE!!! Only use the knowledge I have provided you with.

  DO NOT HALLUCINATE !!!

  Context: {context}
  Question: {question}
  Answer:

  """
  return (llm.invoke(prompt))

In [132]:
print(rag(retrieved_documents, query, llm))

According to the Bharatiya Nyaya Sanhita, Clause 80 is not directly related to Dowry Death.

However, I found that Clause 2 of Section 304B defines "Dowry Death". It states: "Context: otherwise than under normal circumstances within seven years of her marriage and it is shown that soon before her death she was subjected to cruelty or harassment by her husband or any relative of her husband for, or in connection with, any demand for dowry, such death shall be called “dowry death”, and such husband or relative shall be deemed to have caused her death."

Therefore, the relevant clause related to Dowry Death is Section 304B(2) of Bharatiya Nyaya Sanhita.


We can see the answer is not exactly correct. The correct clause is 79(1) and 79(2), as per the document.

## Seeing extracted documents for retrieval using Query Expansion by Answer Generation

In [145]:
query = "Which clause is related to dowry death?"

In [146]:
content = llm.invoke(
f"""
You are a helpful expert legal assistant. Provide an example answer to the given question, that might be found in a document like the official penal code of some country.

Question:
{query}
""")

In [147]:
print(content)

In many countries, including India, Section 304-B of the Indian Penal Code (IPC) deals with the offense of "Dowry Death".

The relevant provision reads:

"304-B. Dowry death: Whoever commits dowry death shall be punished with imprisonment for not less than seven years but which may extend to sentence of imprisonment for life, or with fine, or with both."

This section was introduced by the Dowry Prohibition Act, 1961, to specifically address the growing problem of dowry deaths in India.


In [148]:
joint_query = f"{query} {content}"

In [149]:
results = chroma_collection.query(query_texts=[joint_query], n_results=5)
retrieved_documents = results['documents'][0]

for document in retrieved_documents:
    print((document))
    print('\n')

otherwise than under normal circumstances within seven years of her marriage and it isshown that soon before her death she was subjected to cruelty or harassment by her husbandor any relative of her husband for, or in connection with, any demand for dowry, such deathshall be called “ dowry death ”, and such husband or relative shall be deemed to have causedher death. explanation. — for the purposes of this sub - section, “ dowry ” shall have the same meaning as in section 2 of the dowry prohibition act, 1961. ( 2 ) whoever commits dowry death shall be punished with imprisonment for a term which shall not be less than seven years but which may extend to imprisonment for life. 80. every man who by deceit causes any woman who is not lawfully married to him to believe that she is lawfully married to him and to cohabit or have sexual intercourse with him in that belief, shall be punished with imprisonment of either description for a term which may


extend to ten years, and shall also be li

Even now the relevant texts can't be found.

In [150]:
print(rag(retrieved_documents, joint_query, llm))

According to the Bharatiya Nyaya Sanhita, Clause 79 is related to dowry death. The relevant provision reads: "Clause 79 of the bill seeks to provide punishment for dowry death, which shall be with imprisonment for a term which shall not be less than seven years but which may extend to imprisonment for life."


Results are better.

## Seeing extracted documents for retrieval using Query Expansion by Multiple Queries

In [151]:
query = "Which clause is related to dowry death?"

In [152]:
content = llm.invoke(f"""
            You are a helpful expert legal assistant. Your users are asking questions about the penal code of some country.
            Suggest up to five additional related questions to help them find the information they need, for the provided question.
            Suggest only short questions without compound sentences. Suggest a variety of questions that cover different aspects of the topic.
            Make sure they are complete questions, and that they are related to the original question.
            Output one question per line. Do not number the questions.

            Question by user : {query}
""")

In [153]:
print(content)

What is the definition of dowry in the penal code?

Is there a specific section or article that defines dowry and dowry death?

Can you provide more context about the circumstances surrounding the dowry death, such as the marital status of the victim?

Are there any specific penalties or punishments outlined for those found guilty of committing a dowry death?

Does the penal code specify who is considered responsible for the dowry death, such as the husband or other family members?


In [154]:
augmented_query = f"{query} {content}"

In [155]:
results = chroma_collection.query(query_texts=[augmented_query], n_results=5)
retrieved_documents = results['documents'][0]

for document in retrieved_documents:
    print((document))
    print('\n')

otherwise than under normal circumstances within seven years of her marriage and it isshown that soon before her death she was subjected to cruelty or harassment by her husbandor any relative of her husband for, or in connection with, any demand for dowry, such deathshall be called “ dowry death ”, and such husband or relative shall be deemed to have causedher death. explanation. — for the purposes of this sub - section, “ dowry ” shall have the same meaning as in section 2 of the dowry prohibition act, 1961. ( 2 ) whoever commits dowry death shall be punished with imprisonment for a term which shall not be less than seven years but which may extend to imprisonment for life. 80. every man who by deceit causes any woman who is not lawfully married to him to believe that she is lawfully married to him and to cohabit or have sexual intercourse with him in that belief, shall be punished with imprisonment of either description for a term which may


extend to ten years, and shall also be li

In [158]:
print(rag(retrieved_documents, augmented_query, llm))

According to the provided text, Clause 79 is related to dowry death.

As per Clause 79, "Dowry death" refers to a situation where a woman dies under circumstances that suggest she was subjected to cruelty or harassment by her husband or any relative of her husband, and this abuse was in connection with a demand for dowry.

The definition of dowry is given as follows: "For the purposes of this sub-section, 'dowry' shall have the same meaning as in Section 2 of the Dowry Prohibition Act, 1961".

In terms of the circumstances surrounding the dowry death, the relevant information is that the woman was married and died under circumstances suggesting she was subjected to cruelty or harassment by her husband or his relatives.

The penal code outlines a specific penalty for those found guilty of committing a dowry death: imprisonment for a term which shall not be less than seven years but which may extend to imprisonment for life, with fine.

According to the clause, the husband or relative wh

The output is evidently even better.